# Serag el-dein Amged Abd-Alfattah Ahmed
# 20201498528

In [1]:
from bs4 import BeautifulSoup
import requests 
import re
import pandas as pd

# list from Rotten tomatoes scraping

In [2]:
def urls_scrabing(site):
    urls = []
    soup = BeautifulSoup(site.text, 'html.parser')
    urls_in_html = soup.find('div', class_ = 'panel-body content_body allow-overflow').find('table', class_ = 'table')
    for a in urls_in_html.find_all('a', href = True):
        urls.append('https://www.rottentomatoes.com/' + a['href'])
    return urls

In [3]:
site = requests.get('https://www.rottentomatoes.com/top/bestofrt/')

In [4]:
urls = urls_scrabing(site=site)

# HTML scraping

## Rotten Tomatoes 

In [5]:
df_list = []
for movie_url in urls:
        supsite = requests.get(movie_url)
        
        soup = BeautifulSoup(supsite.text,'html.parser')
        
        title = soup.find('title').contents[0][:-len(" - Rotten Tomatoes")]
        audience_score = str(soup.find('score-board'))[len('<score-board audiencescore="'):len('<score-board audiencescore="')+2]        
        audience_count = soup.find('a', class_ = 'scoreboard__link scoreboard__link--audience').contents[0]
        
        score_board_txt = str(soup.find('score-board'))
        a = score_board_txt.find('tomatometerscore="') + len('tomatometerscore="')
        b = score_board_txt.find('" tomatometerstate="certified-fresh">')
        tomatometer_score = score_board_txt[a:b]
        
        a2 = score_board_txt.find('slot="critics-count">') + len('slot="critics-count">')
        b2 = score_board_txt.find(' Reviews</a>')
        tomatometer_count = score_board_txt[a2:b2]

        df_list.append({'title': title,'audience score': int(audience_score),'audience count':audience_count,'tomatometer score':int(tomatometer_score),'tomatometer_count':tomatometer_count })


In [6]:
df = pd.DataFrame(df_list)

In [13]:
df

,title,audience score,audience count,tomatometer score,tomatometer_count
0,It Happened One Night,93,"25,000+ Ratings",99,104
1,Citizen Kane,90,"100,000+ Ratings",99,125
2,The Wizard of Oz,89,"250,000+ Ratings",98,162
3,Modern Times,95,"25,000+ Ratings",98,111
4,Black Panther,79,"50,000+ Ratings",96,527
...,...,...,...,...,...
95,Never Rarely Sometimes Always,53,"1,000+ Ratings",99,238
96,Baby Driver,86,"50,000+ Ratings",92,397
97,Spider-Man: Homecoming,87,"100,000+ Ratings",92,396
98,"The Godfather, Part II",97,"250,000+ Ratings",96,121


# IMDB Scraping

In [8]:
url2 = 'http://www.imdb.com/chart/top'
response = requests.get(url2)
soup2 = BeautifulSoup(response.text, 'lxml')

In [9]:
movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value') for b in soup.select('td.posterColumn span[name=ir]')]
votes = [b.attrs.get('data-value') for b in soup.select('td.ratingColumn strong')]


In [10]:
# create a empty list for storing
# movie information
list = []

# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):

    # Separating movie into: 'place',
    # 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]

    data = {"movie_title": movie_title,
            "year": year,
            "place": place,
            "star_cast": crew[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    list.append(data)


In [11]:
for movie in list:
    print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +') -', 'Starring:', moive['star_cast'], movie['rating'])


In [12]:
# Downloading imdb top 250 movie's data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

movies = soup.select('td.titleColumn')
links = [a.attrs.get('href') for a in soup.select('td.titleColumn a')]
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]

ratings = [b.attrs.get('data-value')
        for b in soup.select('td.posterColumn span[name=ir]')]

votes = [b.attrs.get('data-value')
        for b in soup.select('td.ratingColumn strong')]

list = []

# create a empty list for storing
# movie information
list = []

# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):

    # Separating movie into: 'place',
    # 'title', 'year'
    movie_string = movies[index].get_text()
    movie = (' '.join(movie_string.split()).replace('.', ''))
    movie_title = movie[len(str(index))+1:-7]
    year = re.search('\((.*?)\)', movie_string).group(1)
    place = movie[:len(str(index))-(len(movie))]
    data = {"movie_title": movie_title,
            "year": year,
            "place": place,
            "star_cast": crew[index],
            "rating": ratings[index],
            "vote": votes[index],
            "link": links[index]}
    list.append(data)

# printing movie details with its rating.
for movie in list:
    print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +
        ') -', 'Starring:', movie['star_cast'], movie['rating'])


1 - The Shawshank Redemption (1994) - Starring: Frank Darabont (dir.), Tim Robbins, Morgan Freeman 9.240009967136304
2 - The Godfather (1972) - Starring: Francis Ford Coppola (dir.), Marlon Brando, Al Pacino 9.160958762781789
3 - The Dark Knight (2008) - Starring: Christopher Nolan (dir.), Christian Bale, Heath Ledger 8.993801667493571
4 - The Godfather: Part II (1974) - Starring: Francis Ford Coppola (dir.), Al Pacino, Robert De Niro 8.990088730566956
5 - 12 Angry Men (1957) - Starring: Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb 8.95074395698382
6 - Schindler's List (1993) - Starring: Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes 8.94003370863721
7 - The Lord of the Rings: The Return of the King (2003) - Starring: Peter Jackson (dir.), Elijah Wood, Viggo Mortensen 8.927385388647123
8 - Pulp Fiction (1994) - Starring: Quentin Tarantino (dir.), John Travolta, Uma Thurman 8.85882664643615
9 - The Lord of the Rings: The Fellowship of the Ring (2001) - Starring: Peter Jackson (dir